<a href="https://colab.research.google.com/github/hieu-le-2412/ai-course/blob/master/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this exercise, you will build a CNN model to classify: horse vs human.

Firstly, you need to run the following code to download the dataset.

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2019-06-19 04:04:14--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 2607:f8b0:4001:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   189MB/s    in 0.8s    

2019-06-19 04:04:15 (189 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2019-06-19 04:04:17--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  --.-KB/s    in 0.09s   

2019-06-19 04:04:17 (120 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



The following python code will use the OS library to use Operating System libraries, giving you access to the file system, and the zipfile library allowing you to unzip the data. 

In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

The contents of the .zip are extracted to the base directory `/tmp/horse-or-human`, which in turn each contain `horses` and `humans` subdirectories.

In short: The training set is the data that is used to tell the neural network model that 'this is what a horse looks like', 'this is what a human looks like' etc. 

One thing to pay attention to in this sample: We do not explicitly label the images as horses or humans. If you remember with the handwriting example earlier, we had labelled 'this is a 1', 'this is a 7' etc.  Later you'll see something called an ImageGenerator being used -- and this is coded to read images from subdirectories, and automatically label them from the name of that subdirectory. So, for example, you will have a 'training' directory containing a 'horses' directory and a 'humans' one. ImageGenerator will label the images appropriately for you, reducing a coding step. 

Let's define each of these directories:

In [0]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/validation-horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/validation-humans')

### Data Preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to `float32` tensors, and feed them (with their labels) to our network. We'll have one generator for the training images and one for the validation images. Our generators will yield batches of images of size 300x300 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network. (It is uncommon to feed raw pixels into a convnet.) In our case, we will preprocess our images by normalizing the pixel values to be in the `[0, 1]` range (originally all values are in the `[0, 255]` range).

In Keras this can be done via the `keras.preprocessing.image.ImageDataGenerator` class using the `rescale` parameter. This `ImageDataGenerator` class allows you to instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs: `fit_generator`, `evaluate_generator`, and `predict_generator`.

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

IMG_WIDTH, IMG_HEIGHT = 299,299

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(IMG_WIDTH, IMG_HEIGHT),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(IMG_WIDTH, IMG_HEIGHT),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Building a Small Model from Scratch

But before we continue, let's start defining the model:





**YOUR TASK**: Construct the  model containing the following layers:
1. A convolution layer: 16 filters of 3x3, activation = relu
2. A max pooling layer: filter of 2x2, stride = 2
3. A convolution layer: 32 filters of 3x3, activation = relu
4. A max pooling layer: filter of 2x2, stride = 2
5. A convolution layer: 64 filters of 3x3, activation = relu
6. A max pooling layer: filter of 2x2, stride = 2
7. A flatten layer
8. A dense layer: 512 neurons, activation = relu
9. A dense layer: 1 neurons, activation = sigmoid



Note that because we are facing a two-class classification problem, i.e. a *binary classification problem*, we will end our network with a [*sigmoid* activation](https://wikipedia.org/wiki/Sigmoid_function), so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).

In [0]:
# Create model of InceptionV3
from keras.applications import InceptionV3;
from keras import callbacks

#Get back the convolutional part of a ResNet50 network trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT,3))

model = Sequential()
model.add(base_model)
#Add the fully-connected layers 
model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='relu', name='fc1'))
model.add(Dense(1, activation='sigmoid', name='predictions'))

# Freeze the layers 
for layer in base_model.layers:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in base_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fa35653b6d8> False
<keras.layers.convolutional.Conv2D object at 0x7fa35653ba20> False
<keras.layers.normalization.BatchNormalization object at 0x7fa35653b668> False
<keras.layers.core.Activation object at 0x7fa35653bdd8> False
<keras.layers.convolutional.Conv2D object at 0x7fa38523cf60> False
<keras.layers.normalization.BatchNormalization object at 0x7fa385217f28> False
<keras.layers.core.Activation object at 0x7fa3851a08d0> False
<keras.layers.convolutional.Conv2D object at 0x7fa3850ec908> False
<keras.layers.normalization.BatchNormalization object at 0x7fa3850cbfd0> False
<keras.layers.core.Activation object at 0x7fa38508b8d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7fa384fd2748> False
<keras.layers.convolutional.Conv2D object at 0x7fa385004438> False
<keras.layers.normalization.BatchNormalization object at 0x7fa384f6f128> False
<keras.layers.core.Activation object at 0x7fa384f6ff28> False
<keras.layers.convolutional.Conv

The model.summary() method call prints a summary of the NN 

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
fc1 (Dense)                  (None, 512)               67109376  
_________________________________________________________________
predictions (Dense)          (None, 1)                 513       
Total params: 88,912,673
Trainable params: 67,109,889
Non-trainable params: 21,802,784
_________________________________________________________________


The "output shape" column shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.

Next, we'll configure the specifications for model training. We will train our model with the `binary_crossentropy` loss, because it's a binary classification problem and our final activation is a sigmoid. (For a refresher on loss metrics, see the [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) We will use the `rmsprop` optimizer with a learning rate of `0.001`. During training, we will want to monitor classification accuracy.

**NOTE**: In this case, using the [RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) is preferable to [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) and [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), also automatically adapt the learning rate during training, and would work equally well here.)

In [0]:
from keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [0]:
reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=2,
    min_lr=0)

eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

callbacks = [reduce_learning, eary_stopping]

### Training
Let's train for 15 epochs -- this may take a few minutes to run.

Do note the values per epoch.

The Loss and Accuracy are a great indication of progress of training. It's making a guess as to the classification of the training data, and then measuring it against the known label, calculating the result. Accuracy is the portion of correct guesses. 

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8,
      callbacks=callbacks)

Epoch 1/15
8/8 [==============================] - 34s 4s/step - loss: 7.0840 - acc: 0.4814 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 6s 781ms/step - loss: 7.4853 - acc: 0.5305 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/15
8/8 [==============================] - 5s 684ms/step - loss: 7.4695 - acc: 0.5315 - val_loss: 7.9712 - val_acc: 0.5000

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 4/15
8/8 [==============================] - 6s 692ms/step - loss: 7.5011 - acc: 0.5295 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/15
8/8 [==============================] - 6s 699ms/step - loss: 7.9512 - acc: 0.5013 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 6/15
8/8 [==============================] - 6s 752ms/step - loss: 8.8440 - acc: 0.4453 - val_loss: 7.9712 - val_acc: 0.5000

Epoch 00006: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 7/15
8/8 [==============================] - 8s 952ms

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(IMG_WIDTH, IMG_HEIGHT))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)/255.0

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")